In [1]:
import tensorflow as tf
import cv2
import imghdr
import os
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [3]:
tf.config.list_physical_devices('GPU')


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# 2. Remove dodgy images

In [ ]:
data_dir = "data"
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [ ]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

# 3. Load Data

In [ ]:
data = tf.keras.utils.image_dataset_from_directory ("data")

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

# 4. Scale Data

In [ ]:
data = data.map(lambda x,y: (x/255, y))

In [ ]:
data.as_numpy_iterator().next()


# 5. Split Data

In [ ]:
data = data.shuffle(buffer_size=len(data))

In [ ]:
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import Sequential

data_dir = "./data"
image_size = (128, 128)

batch_size = 16
validation_split = 0.3
seed = 40

train_data = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=validation_split,
    subset="training",
    seed=seed,
    image_size=image_size,
    batch_size=batch_size
)

val_data = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=validation_split,
    subset="validation",
    seed=seed,
    image_size=image_size,
    batch_size=batch_size,
    shuffle=False
)


data_augmentation = Sequential(
    [
        preprocessing.Rescaling(1./255),
        preprocessing.RandomFlip("horizontal"),
        preprocessing.RandomRotation(0.2),
        preprocessing.RandomZoom(0.2),
    ]
)

raw_train_data = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=validation_split,
    subset="training",
    seed=seed,
    image_size=image_size,
    batch_size=batch_size
)

print(raw_train_data.class_names)


train_data = train_data.map(lambda x, y: (data_augmentation(x, training=True), y))
val_data = val_data.map(lambda x, y: (x / 255, y))


In [ ]:
# Get a batch of data
images, labels = next(iter(train_data))

# Plot the first X images and their labels
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy())
    plt.title(raw_train_data.class_names[labels[i]])
    plt.axis("off")


In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


# 6. Build Deep Learning Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(128,128,3)))

model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(lr=0.0001)
lr_scheduler = ReduceLROnPlateau(factor=0.5, patience=3, min_lr=1e-7)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

logdir = 'logs'
tensorboard_callback = TensorBoard(log_dir=logdir)


num_epochs = 40

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

model_checkpoint = ModelCheckpoint(
    'best_model_07_31.h5',
    monitor='val_loss',
    save_best_only=True
)

hist = model.fit(train_data, epochs=num_epochs, validation_data=val_data, callbacks=[tensorboard_callback, lr_scheduler, early_stopping, model_checkpoint])



In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
model.save('eye_state_detection_modelV1_128x128_even_bigger_data_0731.h5')

# 9. Evaluate

In [ ]:
loss_values = hist.history['loss']
val_loss_values = hist.history['val_loss']

print("Loss values:")
for i, loss in enumerate(loss_values):
    print(f"Epoch {i+1}: {loss}")

print("\nValidation loss values:")
for i, val_loss in enumerate(val_loss_values):
    print(f"Epoch {i+1}: {val_loss}")

fig = plt.figure()
plt.plot(loss_values, color='teal', label='loss')
plt.plot(val_loss_values, color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()


In [ ]:
accuracy_values = hist.history['accuracy']
val_accuracy_values = hist.history['val_accuracy']

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))

# Plot loss
ax1.plot(loss_values, color='teal', label='Training loss')
ax1.plot(val_loss_values, color='orange', label='Validation loss')
ax1.set_title('Training and Validation Loss', fontsize=20)
ax1.legend(loc="upper right")

# Plot accuracy
ax2.plot(accuracy_values, color='blue', label='Training Accuracy')
ax2.plot(val_accuracy_values, color='red', label='Validation Accuracy')
ax2.set_title('Training and Validation Accuracy', fontsize=20)
ax2.legend(loc="lower right")

plt.tight_layout()
plt.show()


In [ ]:
loss_values = hist.history['loss']
val_loss_values = hist.history['val_loss']
accuracy_values = hist.history['accuracy']
val_accuracy_values = hist.history['val_accuracy']

for i, loss in enumerate(loss_values):
    print(f"Epoch {i+1}: {loss}")

for i, val_loss in enumerate(val_loss_values):
    print(f"Epoch {i+1}: {val_loss}")

for i, accuracy in enumerate(accuracy_values):
    print(f"Epoch {i+1}: {accuracy}")

for i, val_accuracy in enumerate(val_accuracy_values):
    print(f"Epoch {i+1}: {val_accuracy}")


In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in val_data.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(pre.result(), re.result(), acc.result())